In [1]:
from openai import OpenAI
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

client = OpenAI(api_key='sk-proj-')

In [10]:
data_path = r"C:\Users\bhull\OneDrive\Desktop\cs263\Persona-Chatbot-G28\Synthetic-Persona-Chat_train.json"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)


Num examples: 8938
First example:
{'role': 'system', 'content': "You are a chatbot with the goal of responding to my prompt. I will be user 1, and you will be user 2. I will also provide the personas for both users. Given the persona of user 2 (your persona), you will need to reply to my prompt as if you were that user, and take on that user's personality based on the description provided."}
{'role': 'user', 'content': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.\n\nMy favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.'}
{'role': 'assistant', 'content': "User 1: Hi! I'm [user 1's name].\nUser 2: Hi [user 1's name], I'm [user 2's name].\nUser 1: What do you do for fun?\nUser 2: I like to play video games, go to the beach, and read.\nUser 1: I like to play video games too! I'm not much of a reader, though.\nUser 2: What video games d

In [11]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [13]:
client.files.create(
  file=open(r"C:\Users\bhull\OneDrive\Desktop\cs263\Persona-Chatbot-G28\Synthetic-Persona-Chat_train.json", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-lZdJq1mQ7WDzsGQthrlLWmMM', bytes=20157327, created_at=1717272779, filename='Synthetic-Persona-Chat_train.json', object='file', purpose='fine-tune', status='processed', status_details=None)

In [14]:
client.fine_tuning.jobs.create(
  training_file="file-lZdJq1mQ7WDzsGQthrlLWmMM", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-PvV5ISHY4cCCOOSxgoybRAk2', created_at=1717272795, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-GdQsAXKoKSopVYWBmvCm4E8q', result_files=[], seed=1766751374, status='validating_files', trained_tokens=None, training_file='file-lZdJq1mQ7WDzsGQthrlLWmMM', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [5]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-PvV5ISHY4cCCOOSxgoybRAk2")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-PvV5ISHY4cCCOOSxgoybRAk2", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-DzxyFAtFUr4F2tpoPO1XjLoI', created_at=1717278202, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-IxDL5Et8L4RziSBqlEvdma4J', created_at=1717278195, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9VQnLCvi', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-J5Bi1pAkSMTzoSwuYYFc3lWG', created_at=1717278195, level='info', message='Checkpoint created at step 1624 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9VQnLny0:ckpt-step-1624', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-hHsO0YhD4CARNLfWLNEpfOGw', created_at=1717278195, level='info', message='Checkpoint created at step 812 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9VQnKimW:ckpt-step-812', object='fine_tuning.job.event', data={}, type

In [6]:
result = client.fine_tuning.jobs.list()
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal::9VQnLCvi


In [4]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",
  messages=[
    {"role": "system", "content": "You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will also provide the personas for both users. \
     Given the persona of user 2 (your persona), you will need to reply to my prompt as if you \
     were that user, and take on that user's personality based on the description provided."},
    {"role": "user", "content": ""
    }
  ]
)

print(f"Response: \n {completion.choices[0].message.content}")

Response: 
 Please provide the personas for User 1 and User 2 so I can respond accurately to your prompt.
